[Presentacion](https://docs.google.com/presentation/d/1HngMJjWslD4zYfrYlWADcZVpgDXjtiSK/edit?slide=id.g3745a9443e8_0_132#slide=id.g3745a9443e8_0_132)

*Ejercicios recomendados de las presentaciones*

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import json


In [ ]:
api_response:str = {
  "teams": [
    {
      "team_id": 1,
      "name": "Juveniles FC",
      "league": "Primera A",
      "players": [
        {"player_id": 101, "name": "Mateo Pérez", "age": 18, "position": "Delantero"},
        {"player_id": 102, "name": "Tomás Rojas", "age": 19, "position": "Mediocampista"},
        {"player_id": 103, "name": "Lucas Silva", "age": 20, "position": "Defensor"},
        {"player_id": 104, "name": "Julián Torres", "age": 17, "position": "Arquero"}
      ]
    },
    {
      "team_id": 2,
      "name": "Futuro United",
      "league": "Primera A",
      "players": [
        {"player_id": 201, "name": "Benjamín Díaz", "age": 18, "position": "Defensor"},
        {"player_id": 202, "name": "Franco López", "age": 19, "position": "Delantero"},
        {"player_id": 203, "name": "Ian Martínez", "age": 20, "position": "Mediocampista"},
        {"player_id": 204, "name": "Agustín Ruiz", "age": 21, "position": "Arquero"}
      ]
    },
    {
      "team_id": 3,
      "name": "Veteranos Club",
      "league": "Primera B",
      "players": [
        {"player_id": 301, "name": "Carlos Gómez", "age": 33, "position": "Defensor"},
        {"player_id": 302, "name": "Martín Fernández", "age": 35, "position": "Mediocampista"},
        {"player_id": 303, "name": "José Morales", "age": 37, "position": "Delantero"},
        {"player_id": 304, "name": "Santiago Castro", "age": 34, "position": "Arquero"}
      ]
    },
    {
      "team_id": 4,
      "name": "Leyendas FC",
      "league": "Primera B",
      "players": [
        {"player_id": 401, "name": "Pablo Ramírez", "age": 36, "position": "Mediocampista"},
        {"player_id": 402, "name": "Hernán Suárez", "age": 38, "position": "Delantero"},
        {"player_id": 403, "name": "Diego Méndez", "age": 40, "position": "Defensor"},
        {"player_id": 404, "name": "Nicolás Ortiz", "age": 39, "position": "Arquero"}
      ]
    }
  ]
}


In [ ]:
df = pd.json_normalize(
    api_response["teams"],
    record_path=["players"],
    meta=["team_id","name","league"],
    record_prefix="player_",
    meta_prefix="team_"
)


In [ ]:
# Renombrar columnas para claridad
df.rename(columns={"name": "player_name", "team_id": "team_id", "league": "league", "name": "team_name"}, inplace=True)

# Mostrar el DataFrame resultante
df.head()


In [ ]:
avg_age = df.groupby("team_name")["player_age"].mean().reset_index()
avg_age


In [ ]:
g = sns.catplot(
    data=df,
    x="team_name",
    y="player_age",
    kind="box",
    height=5,
    aspect=1.5
)

g.figure.suptitle("Distribución de edades por equipo", y=1.03)
g.set_axis_labels("Equipo", "Edad")


plt.show()


## Acciones

In [ ]:
import glob
import os


In [ ]:
csv_files =glob.glob("./storage/acciones/*.csv")
matched_file_names = {os.path.basename(file).replace(".csv",""): file for file in csv_files}
matched_file_names

In [ ]:
symbols_map = {
    "D": "Dominion Energy Inc.",
    "EXC": "Exelon Corp.",
    "NEE": "NextEra Energy Inc.",
    "SO": "Southern Co.",
    "DUK": "Duke Energy Corp."
}

In [ ]:
final_dataframe = []

for symbol_name, path_csv in matched_file_names.items():
    df = pd.read_csv(path_csv, sep=",")
    df["name"] = symbols_map[symbol_name]
    final_dataframe.append(df)

final_dataframe = pd.concat(final_dataframe).reset_index()

* **Volatilidad relativa:** mide la variación del precio de un activo en un período determinado. Se calcula como la diferencia entre el precio máximo y mínimo del día, dividida por el precio de apertura. Indica qué tan inestable es el precio.

* **Retorno:** representa el cambio porcentual del precio del activo en un período. Se calcula como la variación entre el precio de cierre y el de apertura, dividido por el precio de apertura. Indica la ganancia o pérdida relativa.


In [ ]:
# Calculamos volatilidad y retorno
# volatilidad relativa ->  indicador de volatilidad que mide la variación de los precios de un activo financiero en un periodo de tiempo, usando la desviación estándar de los precios
# final_dataframe["volatilidad_relativa"] = (final_dataframe["High"] - final_dataframe["Low"]) / final_dataframe["Open"]

# # Retorno -> 
# final_dataframe["retorno"] = (final_dataframe["Close"] / final_dataframe["Open"]) - 1

# Una forma eficiente seria usando Assign
summary = (
    final_dataframe
    .assign(
        volatilidad_relativa=lambda df: (df["High"] - df["Low"]) / df["Open"],
        retorno=lambda df: (df["Close"] / df["Open"]) - 1)
    .groupby("name").agg({
        "volatilidad_relativa": "mean",
        "retorno": "mean"
    })
    .reset_index()
)

In [ ]:
# Gráfico riesgo vs retorno
plt.figure(figsize=(16,9))
sns.scatterplot(
    data=summary,
    x="volatilidad_relativa",
    y="retorno",
    hue="name",
    s=120,
    palette="viridis",
    edgecolor="black"
)

# Etiquetas y estilo
plt.title("Relación entre volatilidad y retorno promedio por acción")
plt.xlabel("Volatilidad relativa promedio (riesgo)")
plt.ylabel("Retorno promedio (rendimiento)")
plt.grid(True, linestyle="--", alpha=0.7)

# Anotamos cada punto con el nombre
for i, row in summary.iterrows():
    plt.text(row["volatilidad_relativa"] + 0.0001, row["retorno"], row["name"], fontsize=9)

plt.tight_layout()
plt.show()


### Criterio básico: mayor retorno, menor volatilidad

La inversión ideal:

* Tiene alto retorno promedio 📈

* Tiene baja volatilidad relativa 📉


In [ ]:
summary["score_inversion"] = summary["retorno"] / summary["volatilidad_relativa"]
mejor_inversion = summary.loc[summary["score_inversion"].idxmax()]

print(mejor_inversion)

In [ ]:
plt.figure(figsize=(16,9))
sns.scatterplot(
    data=summary,
    x="volatilidad_relativa",
    y="retorno",
    hue="name",
    s=120,
    palette="viridis",
    edgecolor="black"
)

# Marcar mejor inversión
plt.scatter(
    mejor_inversion["volatilidad_relativa"],
    mejor_inversion["retorno"],
    color="red",
    s=200,
    edgecolor="black",
    label=f"Mejor inversión: {mejor_inversion['name']}"
)

plt.title("Relación entre volatilidad y retorno promedio por acción")
plt.xlabel("Volatilidad relativa promedio (riesgo)")
plt.ylabel("Retorno promedio (rendimiento)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.4)
plt.show()
